

        BEGIN TRANSACTION;

        DELETE FROM primary_classifications
        USING primary_classifications_staging
        WHERE primary_classifications.event_id = primary_classifications_staging.event_id;

        INSERT INTO primary_classifications
        SELECT * FROM primary_classifications_staging;

        END TRANSACTION;

        


In [178]:
def query_db():
    """
    Accepts query string and connection
    Uploads result of query into table
    """

    connection = psycopg2.connect(host=HOSTNAME,
                                  user=USERNAME,
                                  password=PASSWORD,
                                  dbname=DATABASE,
                                  port=5439)
    # run query and store results in a dataframe
    LOGGER.info("Running query...")

    cur = connection.cursor()
    
    cur.execute("""SELECT * FROM primary_classifications""")
    column_list = [c[0] for c in cur.description]
    df_query = pd.DataFrame(cur.fetchall(), columns=column_list)
    
    cur.close()
    connection.close()
    
    return df_query


In [179]:
df = query_db()

In [180]:
df.shape

(217329, 4)

In [106]:
df

,event_id,primary_classification,primary_score,uploaded_time
0,5647,birthday,0.103638,2017-07-10 12:11:25.053366
1,8461,birthday,0.538557,2017-07-10 12:11:25.056652
2,6955,birthday,0.607162,2017-07-10 12:11:25.050251
3,4137,birthday,0.363155,2017-07-10 12:11:25.063126
4,3350,birthday,0.167572,2017-07-10 12:11:25.059787


In [87]:
import os
import logging
import psycopg2
import pandas as pd


LOGNAME = 'log/event_classifier_log'
logging.basicConfig(filename=LOGNAME,
                    filemode='a',
                    format='%(asctime)s -  %(name)s - %(levelname)s - %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)
LOGGER = logging.getLogger(__name__)


# connect to old cluster
HOSTNAME = '10.69.10.122'
USERNAME = 'iman'
PASSWORD = os.environ['DATABASEPASSWORD']
DATABASE = 'dev'


def upload_records(query, tup):
    """
    Accepts query string and connection
    Uploads result of query into table
    """

    connection = psycopg2.connect(host=HOSTNAME,
                                  user=USERNAME,
                                  password=PASSWORD,
                                  dbname=DATABASE,
                                  port=5439)
    # run query and store results in a dataframe
    LOGGER.info("Running query...")

    cur = connection.cursor()
    args_str = ','.join(cur.mogrify("(%s,%s,%s,%s)", x).decode() for x in tup)
    print(args_str)
    cur.execute("""
        INSERT INTO primary_classifications_staging (event_id, primary_classification, primary_score, uploaded_time) 
                    VALUES """ + args_str) 

    connection.commit()
    
    cur.close()
    connection.close()


In [88]:
from datetime import datetime

In [169]:
import helpers.query_events

In [89]:
import numpy as np

In [157]:
df = pd.DataFrame(columns=['event_id','primary_classification', 'primary_score', 'upload_time'])
for i in range(1):
    df.loc[i] = [5648, 'birthday',np.random.random(1)[0], datetime.today()]

In [162]:
upsert_list = df.values.tolist()
upsert_list

[[5648,
  'birthday',
  0.5945650812954429,
  Timestamp('2017-07-10 12:45:12.390722')]]

In [167]:
helpers.query_events.create_staging_and_insert(upsert_list)

In [170]:
helpers.query_events.merge_records('primary_classifications')

TypeError: merge_records() takes 0 positional arguments but 1 was given